<a href="https://colab.research.google.com/github/Alecia113/NLP-Lab/blob/main/ex4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

双向RNN
单项RNN 是从前往后把单词复制到序列里面，这样的话对于第i个序列的单词，它只有前i-1个序列的信息，如果我们再加上一个从后往前的序列，第i个单词就同时包含了，0到i-1，以及从i+1 到最后的单词的信息，这样的话对于整个模型来说，就可以更好的理解单词上下文的意思，对于单词表达也会更加清晰一点。

双向RNN增加了后面一些单词的信息，因为单词的上下文都很重要。
比如，the name of your son
如果你从前往后得到the name 这个东西；它是不知道name是有什么含义的
因为name是指你儿子的名字，必须要把后面的意思也理解清楚，这样才能更好的理解the name

In [1]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '16bT0gkTI3EEemyi-6rMaZ-bJHfsCbFGU'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('embedded_docs.pkl')  

id = '1xjgvacGXn5hSWERBw5zTAFtHessESCb-'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('labels.pkl')  

import pickle
input_embeddings = pickle.load(open("embedded_docs.pkl","rb"))
label = pickle.load(open("labels.pkl","rb"))

In [2]:
# Split into training and testing dataset using scikit-learn
# For more details, you can refer to: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

from sklearn.model_selection import train_test_split
train_embeddings, test_embeddings, train_label, test_label = train_test_split(input_embeddings,label,test_size = 0.2, random_state=0)

# The processed data has the shape of (data_size, sequence_length, embedding_dimension)
print(train_embeddings.shape)
print(train_label.shape)

(1805, 512, 25)
(1805,)


In [3]:
import torch
# You can enable GPU here (cuda); or just CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
import torch.nn as nn
import torch.nn.functional as F


class Bi_RNN_Model(nn.Module):
    def __init__(self):
        super(Bi_RNN_Model, self).__init__()
        # Please complete the code for the modeling part
        self.rnn = nn.RNN(n_input, n_hidden, num_layers = 2, batch_first = True, dropout = 0.2, bidirectional = True)
        self.linear = nn.Linear(n_hidden*2, n_class)

    def forward(self, x):        
        # Please complete the code for forward propagation
        rnn_output, h_n = self.rnn(x)
        x_last = torch.cat((h_n[2,:], h_n[3,:]), 1)
        output = self.linear(x_last)
        return output

'''
 C=torch.cat((A,B),0)#按维数0（行）拼接
  C=torch.cat((A,D),1)#按维数1（列）拼接
'''


'\n C=torch.cat((A,B),0)#按维数0（行）拼接\n  C=torch.cat((A,D),1)#按维数1（列）拼接\n'

In [5]:
import numpy as np
import torch.optim as optim

# Please assign values to these variables by using other variables (instead of hard code)
# This should be the length of each sequence
seq_length = 512
# This should be the input feature dimension
n_input = 25
# This should be the number of class
n_class = 4

#Please decide the hyperparameters here by yourself
n_hidden = 64
batch_size = 128
total_epoch = 20
learning_rate = 0.001

In [6]:
from sklearn.metrics import accuracy_score

model = Bi_RNN_Model().to(device)
criterion = nn.CrossEntropyLoss()

# Please find which optimizer provides a higher f1
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

for epoch in range(total_epoch):
    train_loss = 0
    for ind in range(0,train_embeddings.shape[0],batch_size):
        input_batch = train_embeddings[ind:min(ind+batch_size, train_embeddings.shape[0])]
        target_batch = train_label[ind:min(ind+batch_size, train_embeddings.shape[0])]
        input_batch_torch = torch.from_numpy(input_batch).float().to(device)
        target_batch_torch = torch.from_numpy(target_batch).view(-1).to(device)

        model.train()
        optimizer.zero_grad()
        outputs = model(input_batch_torch) 
        loss = criterion(outputs, target_batch_torch)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    print('Epoch: %d, train loss: %.5f'%(epoch + 1, train_loss))

print('Finished Training')

## Prediction
model.eval()
outputs = model(torch.from_numpy(test_embeddings).float().to(device)) 
predicted = torch.argmax(outputs, 1)

# classification_report builds a text report showing the main classification metrics
# The returned report includes the 'weighted avg f1' we want (refer to the sample output)
# More details can be found from: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html
from sklearn.metrics import classification_report
print(classification_report(test_label, predicted.cpu().numpy(),digits=4))

Epoch: 1, train loss: 20.33082
Epoch: 2, train loss: 19.56509
Epoch: 3, train loss: 18.93629
Epoch: 4, train loss: 18.17504
Epoch: 5, train loss: 17.26345
Epoch: 6, train loss: 16.57210
Epoch: 7, train loss: 15.77502
Epoch: 8, train loss: 15.13489
Epoch: 9, train loss: 14.64007
Epoch: 10, train loss: 14.01177
Epoch: 11, train loss: 13.42441
Epoch: 12, train loss: 12.93753
Epoch: 13, train loss: 12.16001
Epoch: 14, train loss: 10.78142
Epoch: 15, train loss: 10.20065
Epoch: 16, train loss: 9.95631
Epoch: 17, train loss: 9.85323
Epoch: 18, train loss: 9.45550
Epoch: 19, train loss: 8.49514
Epoch: 20, train loss: 7.86050
Finished Training
              precision    recall  f1-score   support

           0     0.7396    0.7634    0.7513        93
           1     0.7381    0.7623    0.7500       122
           2     0.6571    0.6330    0.6449       109
           3     0.7440    0.7266    0.7352       128

    accuracy                         0.7212       452
   macro avg     0.7197    0.7